# Training the CHILDES Tokenizer

Using the phonemes in our CHILDES dataset, we train a tokenizer that just splits according to whitespace.

In [8]:
import pandas as pd

from datasets import load_dataset, get_dataset_config_names
from tokenizers import Tokenizer, models, normalizers, pre_tokenizers, trainers, processors, decoders
from transformers import GPT2TokenizerFast

In [9]:
phoible = pd.read_csv('../../../data/phoible.csv')
phoible_phonemes = phoible.Phoneme.unique()

/var/folders/nz/6tzh0bsj2txd1cz18gpcms_c0000gn/T/ipykernel_66936/769076702.py:1: DtypeWarning: Columns (4,7,8,11) have mixed types. Specify dtype option on import or set low_memory=False.
  phoible = pd.read_csv('../../../data/phoible.csv')


In [19]:
MIN_COUNT = 10

def build_vocabulary(datasets, column='phonemized_utterance', allow_non_phoible=False):

    vocab = {'UNK' : 0, 'PAD' : 1, 'WORD_BOUNDARY' : 2, 'UTT_BOUNDARY' : 3}
    unk_tokens = []
    token_counts = {}
    for dataset in datasets:
        for line in dataset[column]:
            tokens = line.strip().split()
            for token in tokens:
                if token not in token_counts:
                    token_counts[token] = 0
                token_counts[token] += 1
        
    # Add tokens to vocab if they are not in phoible and have a count greater than MIN_COUNT
    for token, count in token_counts.items():
        if count > MIN_COUNT and token not in vocab:
            if token not in phoible_phonemes and not allow_non_phoible:
                unk_tokens.append(token)
            else:
                vocab[token] = len(vocab)

    print('Tokens not found in phoible: ', {token: token_counts[token] for token in unk_tokens})
    print('Vocab: ', vocab)
    print('Vocab size: ', len(vocab))
    return vocab

def build_phoneme_tokenizer(vocab):

    tokenizer = Tokenizer(models.WordLevel(vocab=vocab, unk_token='UNK'))
    # tokenizer.normalizer = normalizers.Sequence([normalizers.Replace(' WORD_BOUNDARY', ''), normalizers.Strip()]) 
    tokenizer.normalizer = normalizers.Sequence([normalizers.Strip()]) 
    tokenizer.add_special_tokens(["UNK", "PAD", "UTT_BOUNDARY", "WORD_BOUNDARY"])
    tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()
    tokenizer.post_processor = processors.TemplateProcessing(
        single="UTT_BOUNDARY $A",
        pair="UTT_BOUNDARY $A UTT_BOUNDARY $B:1",
        special_tokens=[("UTT_BOUNDARY", tokenizer.token_to_id("UTT_BOUNDARY"))],
    )

    wrapped_tokenizer = GPT2TokenizerFast(tokenizer_object=tokenizer, bos_token='UTT_BOUNDARY', eos_token='UTT_BOUNDARY', pad_token='PAD', unk_token='UNK')
    return wrapped_tokenizer

# Training a Tokenizer for each language in CHILDES

We create a unique tokenizer for each language, to keep the vocabulary size appropriate for each language. For most languages we remove any tokens not found in Phoible. We do not do this for Mandarin or Cantonese as for these languages we merge the tone marker and preceding vowel into one phoneme, whereas Phoible treats tone markers as independent symbols.

In [25]:
languages = get_dataset_config_names('phonemetransformers/CHILDES')
print('Languages:', languages)
datasets = [load_dataset('phonemetransformers/CHILDES', language, split='train') for language in languages]

Languages: ['English', 'EnglishUK', 'French', 'German', 'Spanish', 'Dutch', 'Mandarin', 'Japanese', 'Cantonese', 'Estonian', 'Croatian', 'Danish', 'Basque', 'Hungarian', 'Turkish', 'Farsi', 'Icelandic', 'Indonesian', 'Irish', 'Welsh', 'Korean', 'Swedish', 'Norwegian', 'Quechua', 'Catalan', 'Italian', 'PortuguesePt', 'PortugueseBr', 'Romanian', 'Serbian', 'Polish']


Generating train split: 2564614 examples [00:12, 211448.74 examples/s]
Generating train split: 2043115 examples [00:09, 212605.69 examples/s]
Generating train split: 721121 examples [00:03, 190377.05 examples/s]
Generating train split: 1525559 examples [00:07, 200322.05 examples/s]
Generating train split: 533308 examples [00:02, 191385.68 examples/s]
Generating train split: 403472 examples [00:01, 230634.64 examples/s]
Generating train split: 530342 examples [00:03, 157772.36 examples/s]
Generating train split: 998642 examples [00:04, 245242.77 examples/s]
Generating train split: 205729 examples [00:01, 181253.25 examples/s]
Generating train split: 186921 examples [00:00, 187165.05 examples/s]
Generating train split: 90992 examples [00:00, 215387.00 examples/s]
Generating train split: 84019 examples [00:00, 229158.89 examples/s]
Generating train split: 71537 examples [00:00, 223192.49 examples/s]
Generating train split: 69690 examples [00:00, 190081.99 examples/s]
Generating train spli

In [26]:
for language, dataset in zip(languages, datasets):
    print(f'\nTraining tokenizer for {language}...')
    allow_non_phoible = language in ['Mandarin', 'Cantonese'] # For Mandarin and Cantonese, allow non-phoible tokens since we merge tone with vowels
    vocab = build_vocabulary([dataset], allow_non_phoible=allow_non_phoible)
    tokenizer = build_phoneme_tokenizer(vocab)
    tokenizer.push_to_hub(f"phonemetransformers/CHILDES-{language}-phoneme-tokenizer")
    print(f'Tokenizer for {language} pushed to the hub.')

print(f'\nTrainking tokenizer for all languages...')
vocab = build_vocabulary(datasets)
tokenizer = build_phoneme_tokenizer(vocab)
tokenizer.push_to_hub("phonemetransformers/CHILDES-phoneme-tokenizer")
print('Done.')



Training tokenizer for English...
Tokens not found in phoible:  {}
Vocab:  {'UNK': 0, 'PAD': 1, 'WORD_BOUNDARY': 2, 'UTT_BOUNDARY': 3, 'd̠ʒ': 4, 'ʌ': 5, 's': 6, 't': 7, 'l': 8, 'aɪ': 9, 'k': 10, 'j': 11, 'ʊ': 12, 'ɹ': 13, 'b': 14, 'æ': 15, 'h': 16, 'oʊ': 17, 'm': 18, 'iː': 19, 'ð': 20, 'ɛ': 21, 'z': 22, 'f': 23, 'eɪ': 24, 'w': 25, 'ɪ': 26, 'ɡ': 27, 'ɑ': 28, 'ə': 29, 'p': 30, 'uː': 31, 'i': 32, 'θ': 33, 'ŋ': 34, 'ɔ': 35, 'ɔɪ': 36, 'n': 37, 'd': 38, 'aʊ': 39, 'v': 40, 'ɜː': 41, 't̠ʃ': 42, 'ʃ': 43, 'iə': 44, 'ʒ': 45, 'x': 46}
Vocab size:  47


No files have been modified since last commit. Skipping to prevent empty commit.


Tokenizer for English pushed to the hub.

Training tokenizer for EnglishUK...
Tokens not found in phoible:  {}
Vocab:  {'UNK': 0, 'PAD': 1, 'WORD_BOUNDARY': 2, 'UTT_BOUNDARY': 3, 'ð': 4, 'æ': 5, 'tʰ': 6, 'ɡ': 7, 'ʊ': 8, 'd': 9, 'ɑː': 10, 'l': 11, 'ɪ': 12, 'n': 13, 'eɪ': 14, 't̠ʃ': 15, 'w': 16, 'ɒ': 17, 'ʌ': 18, 'z': 19, 'm': 20, 'iː': 21, 'aɪ': 22, 'h': 23, 'e': 24, 'kʰ': 25, 's': 26, 'ə': 27, 'ɔː': 28, 'ɹ': 29, 'i': 30, 'əʊ': 31, 'uː': 32, 'j': 33, 'ɪə': 34, 'ɔɪ': 35, 'v': 36, 'f': 37, 'ɜː': 38, 'b': 39, 'pʰ': 40, 'd̠ʒ': 41, 'ɐ': 42, 'eə': 43, 'ʃ': 44, 'θ': 45, 'ŋ': 46, 'aʊ': 47, 'ʊə': 48, 'n̩': 49, 'ʒ': 50}
Vocab size:  51


No files have been modified since last commit. Skipping to prevent empty commit.


Tokenizer for EnglishUK pushed to the hub.

Training tokenizer for French...
Tokens not found in phoible:  {}
Vocab:  {'UNK': 0, 'PAD': 1, 'WORD_BOUNDARY': 2, 'UTT_BOUNDARY': 3, 'm': 4, 'a': 5, 'ɑ̃': 6, 'd': 7, 'ɔ': 8, 'n': 9, 'b': 10, 'ʁ': 11, 'ə': 12, 'ɡ': 13, 'ʒ': 14, 'i': 15, 'v': 16, 't': 17, 'k': 18, 'o': 19, 'ɛ̃': 20, 'w': 21, 'y': 22, 'j': 23, 'e': 24, 'ɔ̃': 25, 'p': 26, 'ɛ': 27, 'f': 28, 's': 29, 'z': 30, 'l': 31, 'u': 32, 'ʃ': 33, 'œ': 34, 'ø': 35, 'ɲ': 36, 't̠ʃ': 37, 'd̠ʒ': 38}
Vocab size:  39


...

Tokenizer for Turkish pushed to the hub.

Training tokenizer for Farsi...
Tokens not found in phoible:  {}
Vocab:  {'UNK': 0, 'PAD': 1, 'WORD_BOUNDARY': 2, 'UTT_BOUNDARY': 3, 'm': 4, 'a̟': 5, 'b': 6, 's': 7, 'e': 8, 'r': 9, 'j': 10, 'h': 11, 't̠ʃ': 12, 'kʰ': 13, 'd̪': 14, 'n̪': 15, 'z': 16, 'ʃ': 17, 'ɡ': 18, 'i': 19, 'u': 20, 'o': 21, 'f': 22, 't̪ʰ': 23, 'ɑ': 24, 'd̠ʒ': 25, 'v': 26, 'pʰ': 27, 'l': 28, 'w': 29, 'ɢ': 30}
Vocab size:  31


No files have been modified since last commit. Skipping to prevent empty commit.


Tokenizer for Farsi pushed to the hub.

Training tokenizer for Icelandic...
Tokens not found in phoible:  {}
Vocab:  {'UNK': 0, 'PAD': 1, 'WORD_BOUNDARY': 2, 'UTT_BOUNDARY': 3, 'pʰ': 4, 'iː': 5, 'i': 6, 'aː': 7, 'r̥': 8, 'ɪ': 9, 'ɛ': 10, 't̪ʰ': 11, 's̺': 12, 'j': 13, 'ä': 14, 'k': 15, 'ʋ': 16, 'ɛː': 17, 'r': 18, 'ei̯': 19, 'θ̻': 20, 'l': 21, 'n̪': 22, 't̪': 23, 'ɬ': 24, 'uː': 25, 'ð̺̞': 26, 'ɡ': 27, 'c': 28, 'h': 29, 'ɔ': 30, 'n̪̥': 31, 'äu̯': 32, 'ŋ̥': 33, 'ʏ': 34, 'm': 35, 'f': 36, 'ɔː': 37, 'x': 38, 'cʰ': 39, 'ou̯': 40, 'p': 41, 'ŋ': 42, 'øɪ̯': 43, 'äi̯': 44, 'ɰ': 45, 'ʏː': 46, 'u': 47, 'ɪː': 48, 'œ': 49, 'ç': 50, 'ə': 51, 'œː': 52, 'ɲ': 53, 'm̥': 54, 'ɔi̯': 55, 'z': 56, 'ɲ̥': 57}
Vocab size:  58


No files have been modified since last commit. Skipping to prevent empty commit.


Tokenizer for Icelandic pushed to the hub.

Training tokenizer for Indonesian...
Tokens not found in phoible:  {}
Vocab:  {'UNK': 0, 'PAD': 1, 'WORD_BOUNDARY': 2, 'UTT_BOUNDARY': 3, 's': 4, 'i': 5, 'n': 6, 'm': 7, 'a': 8, 'j': 9, 'u': 10, 'k': 11, 'o': 12, 'h': 13, 'l': 14, 't': 15, 'w': 16, 'd̠ʒ': 17, 'ŋ': 18, 'ə': 19, 'd': 20, 'p': 21, 'ɡ': 22, 'b': 23, 'r': 24, 'ɲ': 25, 't̠ʃ': 26, 'f': 27, 'z': 28, 'ʃ': 29, 'x': 30}
Vocab size:  31
Tokenizer for Indonesian pushed to the hub.

Training tokenizer for Irish...
Tokens not found in phoible:  {}
Vocab:  {'UNK': 0, 'PAD': 1, 'WORD_BOUNDARY': 2, 'UTT_BOUNDARY': 3, 'kʰ': 4, 'a': 5, 'ɾ̪ʲ': 6, 'd̪ˠ': 7, 'eː': 8, 'ʃ': 9, 'ɪ': 10, 'n̪ˠ': 11, 'ə': 12, 'w': 13, 'l̪ˠ': 14, 'ɛ̝': 15, 'ɡ': 16, 'ɾ̪ˠ': 17, 'mˠ': 18, 'x': 19, 'iː': 20, 'sˠ': 21, 'bˠ': 22, 'pˠʰ': 23, 't̪ʲʰ': 24, 'ɔ̝': 25, 'cʰ': 26, 't̪ˠʰ': 27, 'h': 28, 'vˠ': 29, 'ʊ': 30, 'j': 31, 'oː': 32, 'ɑː': 33, 'fˠ': 34, 'd̠ʒ': 35, 'l̪ʲ': 36, 'iːə': 37, 'uːe': 38, 'uː': 39, 'n̪ʲ': 40, 'd̪ʲ': 41, 'ɐ'

# BPE Tokenizers for CHILDES

In [13]:
dataset = load_dataset('phonemetransformers/CHILDES', 'English', split='train')
tokenizer = Tokenizer(models.BPE())
tokenizer.normalizer = normalizers.Sequence(
        [normalizers.NFD(),
         normalizers.Lowercase(),
         normalizers.Strip(),
         normalizers.StripAccents(),
        ]
    )
tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=True)

trainer = trainers.BpeTrainer(vocab_size=8192, special_tokens=["UTT_BOUNDARY", "PAD", "UNK"])
tokenizer.train_from_iterator(dataset['processed_gloss'], trainer=trainer)

tokenizer.decoder = decoders.ByteLevel()
tokenizer.post_processor = processors.TemplateProcessing(
    single="UTT_BOUNDARY $A",
    pair="UTT_BOUNDARY $A UTT_BOUNDARY $B:1",
    special_tokens=[("UTT_BOUNDARY", tokenizer.token_to_id("UTT_BOUNDARY"))],
)

In [14]:
example = dataset['processed_gloss'][300]
encoding = tokenizer.encode(example)
print(f'Example: {example}')
print(encoding.tokens)

Example: is that what you saw?
['UTT_BOUNDARY', 'Ġis', 'Ġthat', 'Ġwhat', 'Ġyou', 'Ġsaw', '?']


In [15]:
wrapped_tokenizer = GPT2TokenizerFast(tokenizer_object=tokenizer, pad_token='PAD', unk_token='UNK', bos_token='UTT_BOUNDARY', eos_token='UTT_BOUNDARY', add_prefix_space=True)
wrapped_tokenizer.push_to_hub("phonemetransformers/CHILDES-English-BPE-gloss-tokenizer")

CommitInfo(commit_url='https://huggingface.co/phonemetransformers/CHILDES-English-BPE-gloss-tokenizer/commit/dc70201e9f3dc609aea522ae4df6cc435f07a55e', commit_message='Upload tokenizer', commit_description='', oid='dc70201e9f3dc609aea522ae4df6cc435f07a55e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/phonemetransformers/CHILDES-English-BPE-gloss-tokenizer', endpoint='https://huggingface.co', repo_type='model', repo_id='phonemetransformers/CHILDES-English-BPE-gloss-tokenizer'), pr_revision=None, pr_num=None)

In [16]:
tokenized = wrapped_tokenizer(example, padding='max_length', max_length=20, truncation=True, add_special_tokens=True)
tokenized

{'input_ids': [0, 115, 92, 95, 67, 781, 8, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}

In [17]:
wrapped_tokenizer.convert_ids_to_tokens(tokenized['input_ids'])

['UTT_BOUNDARY',
 'Ġis',
 'Ġthat',
 'Ġwhat',
 'Ġyou',
 'Ġsaw',
 '?',
 'PAD',
 'PAD',
 'PAD',
 'PAD',
 'PAD',
 'PAD',
 'PAD',
 'PAD',
 'PAD',
 'PAD',
 'PAD',
 'PAD',
 'PAD']

In [18]:
wrapped_tokenizer('this is a test .')

{'input_ids': [0, 124, 115, 61, 3630, 45, 6], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}